## All necessary imports

In [1]:
!pip install googletrans==4.0.0-rc1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
import re
import random
from datasets import load_dataset, Audio, DatasetDict
from collections import Counter
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.metrics import f1_score
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import AUC


from sklearn.preprocessing import LabelEncoder
from googletrans import Translator
from IPython.display import clear_output
from transformers import TrainerCallback
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import tempfile

from IPython.display import Audio as IPythonAudio
import librosa
from transformers import pipeline, TrainingArguments, Trainer, AutoModelForSequenceClassification
from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification
from torch.utils.data import Dataset as TorchDataset
from transformers import DataCollatorWithPadding
import traceback
from datasets import Dataset as hgdataset

from transformers import TFBertModel, TFWav2Vec2Model
from transformers import TFDistilBertModel
from tensorflow.keras.layers import *
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.models import Model


from transformers.utils import logging
logging.set_verbosity_error()
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=dbff860f1e1d4bfa94cd784a005725d91eb7f10842233eb3da92892aa72062bb
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
     

## Text file loading

In [2]:
df_telugu = pd.read_excel("/kaggle/input/shared-task-hunt/Train set/telugu/text/TE-AT-train.xlsx")

df_test_telugu = pd.read_excel("/kaggle/input/shared-task-hunt/Test set/telugu/text/TE-AT-test.xlsx")

In [3]:
df_telugu.rename(columns={'File_Name': 'File Name'}, inplace=True)
df_test_telugu.rename(columns={'File_Name': 'File Name'}, inplace=True)

## Audio file loading

In [4]:
tl_audio_dir = "/kaggle/input/shared-task-hunt/Train set/telugu/audio"
tl_audio_test_dir = "/kaggle/input/shared-task-hunt/Test set/telugu/audio"
tl_audio_files = [os.path.join(tl_audio_dir, file) for file in os.listdir(tl_audio_dir) if file.endswith(".wav")]
tl_audio_test_files = [os.path.join(tl_audio_test_dir, file) for file in os.listdir(tl_audio_test_dir) if file.endswith(".wav")]
tl_file_name = [os.path.join("", file) for file in os.listdir(tl_audio_dir) if file.endswith(".wav")]

all_audio_files = tl_audio_files
all_file_name = tl_file_name
all_audio_test_file = tl_audio_test_files

## Checking the dataset

In [5]:
df_train = df_telugu.copy()
df_test = df_test_telugu.copy()
print(f" The size of the Total dataset {df_train.shape}")
print(f" The size of the Test dataset {df_test.shape}")

 The size of the Total dataset (556, 3)
 The size of the Test dataset (50, 2)


In [6]:
df_train.head()


,Class Label Short,File Name,Transcript
0,R,H_TE_001_R_F_015_001,ఈ కాలంలో మీరు ఒకసారి ఒబ్సర్వ్ చేయండి మన స్టేటు...
1,R,H_TE_001_R_F_015_002,హిందూ థర్మాన్ని ఎవరేమన్నా కూడా వాడికొచ్చే ఒకేఒ...
2,R,H_TE_001_R_F_015_003,ఒక ముస్లింనిగాని. ఒక్క నిమిషం భార్గవి కల్యాణిగ...
3,R,H_TE_001_R_F_015_004,ఈ సెక్యులర్ ఇండియా డెమోక్రాటిక్ ఇండియా అని మాట...
4,R,H_TE_001_R_F_015_005,"హిందువులున్న ఈ భారతదేశంలో ,ఈ భారతదేశంలో , సనాత..."


In [7]:
df_test.sample(5)

,File Name,Transcript
40,TE_TE_041,మీక్ తెల్సా ప్రపంచవ్యాప్తంగా ముస్లింలు అత్యంత ...
30,TE_TE_031,చంద్రబాబు నాయుడుకి చిన్న మెదడు చితికిపోయింది అ...
47,TE_TE_048,"నేన్ నీకొకటే చెప్తున్నా, ధర్మమైన సనాతన ధర్మమైన..."
15,TE_TE_016,కొజ్జాలని చుస్తే ఈ షోప్స్ ట్రాఫిక్ సిగ్నల్స్ ఏ...
6,TE_TE_007,ఎవరు మాత్రం SC కులంలో పుట్టాలని కోరుకుంటారు డబ...


In [8]:
print(all_audio_files[0])
print(all_file_name[0])
print(all_audio_test_file[0])

/kaggle/input/shared-task-hunt/Train set/telugu/audio/H_TE_001_R_F_015_007.wav
H_TE_001_R_F_015_007.wav
/kaggle/input/shared-task-hunt/Test set/telugu/audio/TE_TE_048.wav


In [9]:
text_file_name = list(df_train['File Name'])
only_file_name = [file.split('/')[7][:-4] for file in all_audio_files]

print(text_file_name[0])
print(only_file_name[0])

H_TE_001_R_F_015_001
H_TE_001_R_F_015_007


## Checking for missing files in audio data

In [10]:
for text, audio in zip(text_file_name, only_file_name):
    if(text not in only_file_name):
        print(text)

for text, audio in zip(text_file_name, only_file_name):
    if(audio not in text_file_name):
        print(audio)

for text in text_file_name:
    if(text not in only_file_name):
        df_train.drop(df_train[df_train['File Name'] == text].index, inplace = True)

H_TE_002_G_F_041_005
H_TE_002_G_F_041_006
H_TE_002_G_F_042_001
H_TE_002_G_F_043_002
H_TE_002_G_F_043_003


## Using back translation for data augmentation

In [ ]:
translator = Translator()
def back_translate(text, src_lang, intermediate_lang="en"):
    try:
        translated = translator.translate(text, src=src_lang, dest=intermediate_lang).text
        back_translated = translator.translate(translated, src=intermediate_lang, dest=src_lang).text
        return back_translated
    except Exception as e:
        print(f"Error in back translation: {e}")
        return text

classlist = ['C', 'G', 'R', 'P']

target = df_train['Class Label Short'].value_counts()['N']
augmented_texts = []

for x in classlist:
    count = df_train['Class Label Short'].value_counts()[x]
    tmp = target - count
    start = 0
    if(x == 'N'):
        continue
    while(count != target):
        for index, row in df_train[df_train['Class Label Short'] == x].iterrows():
            original_text = row["Transcript"]
            language = row['File Name'].split('_')[1].lower()
            
            back_translated = back_translate(original_text, src_lang=language)

            augmented_texts.append({"Class Label Short": row['Class Label Short'], 
                                    "File Name": row['File Name'],
                                    "Transcript": back_translated})
            count = count + 1
            start = start + 1
            print(f"{round(start/tmp*100, 2)} of {x} {count}")
            clear_output(wait=True)
            if(count == target):
                break


augmented_df = pd.DataFrame(augmented_texts)
df_train = pd.concat([df_train, augmented_df], ignore_index = True)

11.84 of C 131


In [ ]:
print(f"The new shape of train data after augmentation: {df_train.shape}")
print("Balanaced classes!")
print(df_train['Class Label Short'].value_counts())

In [ ]:
all_audio_files = []

for row in df_train.iterrows():
    
    lang = 'telugu'
    file_name = row[1][1]
    
    directory = '/kaggle/input/shared-task-hunt/Train set/' + lang + '/audio/' + file_name + '.wav'
    all_audio_files.append(directory)

print("Getting the audio file names after augmentation...")
print(f"Number of audios after augmentation: {len(all_audio_files)}")
print(f"Sample of audio file names: {all_audio_files[0]}")

## Label Encoding

In [ ]:
le = LabelEncoder()
df_train['Class Label Short'] = le.fit_transform(df_train['Class Label Short'])
label = list(df_train['Class Label Short'])

encoding_dict = {class_label: index for index, class_label in enumerate(le.classes_)}
print(encoding_dict)

In [ ]:
df_train.drop(columns = ['File Name'], inplace = True)

## Loading audio files

In [ ]:
data_dict = {"audio": all_audio_files, "label": label}
dataset = hgdataset.from_dict(data_dict)
dataset


data_dict = {"audio": all_audio_test_file}
test_dataset_audio = hgdataset.from_dict(data_dict)
test_dataset_audio

audio_sample = dataset[1]
print(f"audio sample details: \n \n {audio_sample}")

dataset = dataset.cast_column("audio",
     Audio(sampling_rate=16_000))
print(f"Training set sample {dataset[3]}\n \n")

test_dataset_audio = test_dataset_audio.cast_column("audio",
     Audio(sampling_rate=16_000))
print(f"Test set sample {test_dataset_audio[3]}")

## Audio Augmentation

In [ ]:
def custom_time_stretch(audio, rate=1.2):
    """Stretch the audio in time."""
    return librosa.effects.time_stretch(y = audio, rate = rate)

def custom_pitch_shift(audio, sampling_rate, n_steps=2):
    """Shift the pitch of the audio."""
    return librosa.effects.pitch_shift(y=audio, sr=sampling_rate, n_steps=n_steps)

def add_white_noise(audio, noise_factor=0.005):
    """Add random white noise to the audio."""
    noise = np.random.randn(len(audio))
    return audio + noise_factor * noise

def change_volume(audio, factor=1.2):
    """Change the volume of the audio."""
    return audio * factor
    
def augment_audio(audio, sampling_rate):
    """Apply a series of augmentations to the audio."""
    # Randomly choose augmentations
    aug_choice = random.choice([custom_time_stretch, custom_pitch_shift, add_white_noise, change_volume])
    
    if aug_choice == custom_time_stretch:
        return custom_time_stretch(audio)
    elif aug_choice == custom_pitch_shift:
        return custom_pitch_shift(audio, sampling_rate)
    elif aug_choice == add_white_noise:
        return add_white_noise(audio)
    elif aug_choice == change_volume:
        return change_volume(audio)

In [ ]:
wav2vec2_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
MAX_LENGTH = 16000

def extract_mfcc(audio, sampling_rate=16000, n_mfcc=13, hop_length=512):
    mfcc = librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=n_mfcc, hop_length=hop_length)
    return mfcc.T

def preprocess_function_wav2vec2(batch, train = True):
    audio = batch["audio"]["array"]
    sampling_rate = batch["audio"]["sampling_rate"]

    if(train):
        label = batch["label"]

    if(train):
        augmented_audio = augment_audio(audio, sampling_rate)
    
    mfcc_features = extract_mfcc(audio, sampling_rate=sampling_rate)

    max_frames = MAX_LENGTH // 512
    if mfcc_features.shape[0] > max_frames:
        mfcc_features = mfcc_features[:max_frames, :]
    else:
        mfcc_features = np.pad(mfcc_features, ((0, max_frames - mfcc_features.shape[0]), (0, 0)), "constant")


    if len(audio) > MAX_LENGTH:
        audio = audio[:MAX_LENGTH]
    else:
        audio = np.pad(audio, (0, MAX_LENGTH - len(audio)), "constant")

    inputs = wav2vec2_processor(audio, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

    batch["input_values"] = inputs["input_values"][0].numpy()
    batch["mfcc_features"] = mfcc_features
    if(train):
        batch["labels"] = label

    return batch

try:
    processed_dataset_wav2vec2 = dataset.map(preprocess_function_wav2vec2)
except:
    print("file not found")

## Splitting the dataset

In [ ]:
split_dataset = processed_dataset_wav2vec2.train_test_split(test_size=0.2, seed=42)
split_dataset = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"],
})

print(split_dataset)

from torch.utils.data import Dataset

class AudioDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        input_values = torch.tensor(item["input_values"], dtype=torch.float32)
        labels = torch.tensor(item["labels"], dtype=torch.long)
        return {"input_values": input_values, "labels": labels}

train_dataset = AudioDataset(split_dataset["train"])
eval_dataset = AudioDataset(split_dataset["validation"])

## Getting unimodals for multimodal

In [ ]:
bert_model = TFBertModel.from_pretrained("bert-base-multilingual-cased")
wav2vec_model = TFWav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

## Callback function for f-1 scores

In [ ]:
class MacroF1Callback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, validation_data=None):
        super(MacroF1Callback, self).__init__()
        self.train_data = train_data
        self.validation_data = validation_data
        self.train_f1_scores = []
        self.val_f1_scores = []
        
    def on_epoch_end(self, epoch, logs={}):
        # Calculate F1 score for training data
        train_pred = np.argmax(self.model.predict(self.train_data[0]), axis=1)
        train_true = self.train_data[1]
        train_f1 = f1_score(train_true, train_pred, average='macro')
        self.train_f1_scores.append(train_f1)
        
        # Calculate F1 score for validation data if provided
        if self.validation_data is not None:
            val_pred = np.argmax(self.model.predict(self.validation_data[0]), axis=1)
            val_true = self.validation_data[1]
            val_f1 = f1_score(val_true, val_pred, average='macro')
            self.val_f1_scores.append(val_f1)
            print(f'\nEpoch {epoch+1}:')
            print(f'Training Macro F1-score: {train_f1:.4f}')
            print(f'Validation Macro F1-score: {val_f1:.4f}')
        else:
            print(f'\nEpoch {epoch+1}:')
            print(f'Training Macro F1-score: {train_f1:.4f}')

## Getting input IDs and mask attention of text

In [ ]:
def preprocess_text(texts):
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors="tf"
    )
    return tokenized["input_ids"], tokenized["attention_mask"]

df_train["Transcript"] = df_train["Transcript"].fillna("").astype(str)
text_input_ids, text_attention_mask = preprocess_text(df_train["Transcript"].tolist())
text_ids_input = Input(shape=(512,), dtype=tf.int32, name="text_ids_input")
text_mask_input = Input(shape=(512,), dtype=tf.int32, name="text_mask_attention")

## Getting text features from mBERT

In [ ]:
def bert_model_wrapper(inputs):
    input_ids, attention_mask = inputs
    return bert_model(input_ids=input_ids, attention_mask=attention_mask).pooler_output
text_features = Lambda(bert_model_wrapper, name="bert_embedding", output_shape=(768,))([text_ids_input, text_mask_input])
text_dense = Dense(256, activation="relu", name="text_dense")(text_features)

## Getting Audio Features

In [ ]:
audio_input = Input(shape=(16000,), dtype=tf.float32, name="audio_input")
mfcc_features_input = Input(shape=(31, 13), dtype=tf.float32, name="mfcc_input")

def get_wav2vec_output(x):
    return wav2vec_model(x).last_hidden_state

wav2vec_output = Lambda(
    get_wav2vec_output,
    output_shape=(None, 768),
    name="wav2vec"
)(audio_input)

wav2vec_flattened = Lambda(lambda x: x[:, 0, :], name="wav2vec_flattened")(wav2vec_output)
mfcc_flattened = Flatten(name="mfcc_flattened")(mfcc_features_input)
combined_audio_features = Concatenate(name="concat_audio_features")([wav2vec_flattened, mfcc_flattened])
audio_dense = Dense(256, activation="relu", name="audio_dense")(combined_audio_features)

## Late Fusion

In [ ]:
combined_features = Concatenate(name="concat_features")([audio_dense, text_dense])

## Classification Head

In [ ]:
x = Dense(128, activation="relu", name="fc1")(combined_features)
x = Dropout(0.5, name="dropout")(x)
output = Dense(5, activation="softmax", name="output")(x)

## Multimodal

In [ ]:
multimodal_model = Model(
    inputs=[audio_input, mfcc_features_input, text_ids_input, text_mask_input], 
    outputs=output
)

multimodal_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
multimodal_model.summary()

## Getting the train & validation data ready

In [ ]:
train_inputs = {
    "audio_input": np.stack([x["input_values"] for x in processed_dataset_wav2vec2]),
    "mfcc_input": np.stack([x["mfcc_features"] for x in processed_dataset_wav2vec2]),
    "text_ids_input": text_input_ids,
    "text_mask_attention": text_attention_mask,
}

train_labels = np.array(label)

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_idx, val_idx in splitter.split(train_inputs["audio_input"], train_labels):
    train_idx = tf.convert_to_tensor(train_idx)
    val_idx = tf.convert_to_tensor(val_idx)
    train_inputs_final = {
        key: tf.gather(value, train_idx) for key, value in train_inputs.items()
    }
    val_inputs = {
        key: tf.gather(value, val_idx) for key, value in train_inputs.items()
    }
    train_labels_final = tf.gather(train_labels, train_idx)
    val_labels = tf.gather(train_labels, val_idx)

In [ ]:
print("Training set sizes:")
for key, value in train_inputs_final.items():
    print(f"{key}: {value.shape}")
print(f"Training labels: {train_labels_final.shape}")

print("\nValidation set sizes:")
for key, value in val_inputs.items():
    print(f"{key}: {value.shape}")
print(f"Validation labels: {val_labels.shape}")

## Training

In [ ]:
f1_callback = MacroF1Callback(
    train_data=(train_inputs_final, train_labels_final),
    validation_data=(val_inputs, val_labels)
)

multimodal_model.fit(
    train_inputs_final,
    train_labels_final,
    batch_size=32,
    epochs=20,
    validation_data=(val_inputs, val_labels),
    callbacks=[f1_callback]
)

## Getting the test data ready

In [ ]:
test_text_input_ids, test_text_attention_mask = preprocess_text(df_test["Transcript"].tolist())
test_processed_dataset_wav2vec2 = test_dataset_audio.map(
    preprocess_function_wav2vec2,
    fn_kwargs={'train': False}
)

test_inputs = {
    "audio_input": np.stack([x["input_values"] for x in test_processed_dataset_wav2vec2]),
    "mfcc_input": np.stack([x["mfcc_features"] for x in test_processed_dataset_wav2vec2]),
    "text_ids_input": test_text_input_ids,
    "text_mask_attention": test_text_attention_mask,
}

predictions = multimodal_model.predict(test_inputs)

predicted_classes = np.argmax(predictions, axis=1)

prediction_probabilities = predictions

In [ ]:
class_mapping = {
    0: 'C',
    1: 'G',
    2: 'N',
    3: 'P',
    4: 'R'
}

## Getting the TSV

In [ ]:
predicted_letters = [class_mapping[pred] for pred in predicted_classes]

predictions_df = pd.DataFrame({
    'Predicted_Class': predicted_letters
})
predictions_df.to_csv('predictions.tsv', sep='\t', index=False)